In [ ]:
from PIL import Image
import os
import sys
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy import misc
from sklearn.cluster import KMeans
from sklearn.externals import joblib

In [ ]:
# ----------------------------------------------------------------------------------
# Name:        Kmeans_rr_1418.ipynb
# Purpose:     Calculate Kmeans clusters from xGeo precip maps between 2014 and 2018.
#
# Author:      hvtola
#
# Created:     11.07.2019
# ----------------------------------------------------------------------------------

In [ ]:
# Working directory
os.chdir("C:\\Metdata\\rr\\1418\\")

# Create list with all filenames that should be processed
rg = os.listdir()

In [ ]:
# Rescale images

def down_res(filename, scale_percent):
    img = cv2.imread(filename)
    # scale_percent = 50 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    # print('Resized Dimensions : ',resized.shape)

    # cv2.imshow("Resized image", resized)
    
    img = Image.fromarray(resized)
    
    return img

In [ ]:
for i in rg:
    img = down_res(i, 50)
    img.save(i)

In [ ]:
# Remove Parts of Image such as legend and dates

def rpoi(filename):
    # original image
    # -1 loads as-is so if it will be 3 or 4 channel as the original
    image = cv2.imread(filename, -1)
    # mask defaulting to black for 3-channel and transparent for 4-channel
    # (of course replace corners with yours)
    mask = np.zeros(image.shape, dtype=np.uint8)
    roi_corners = np.array([[(250,137), (425,0), (597,0), (597,200), (300,465), (300,777), (0,775), (0, 350)]], dtype=np.int32)
    # fill the ROI so it doesn't get wiped out when the mask is applied
    channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,)*channel_count
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)

    # apply the mask
    masked_image = cv2.bitwise_and(image, mask)
    im = Image.fromarray(masked_image)
    
    return im

In [ ]:
# NB! No other files than the files to be processed can be stored in the folder. E.g. .ovr files created when opening a .png in ArcMap will cause error. 
for i in rg:
    im = rpoi(i)
    im.save(i)

In [ ]:
# Converts image into 1 bit data for fast processing
def oneband(filename):
    img = Image.open(filename)
    ur = img.convert(mode='1', matrix=None, dither=None, palette=0, colors=256)
    
    return ur

In [ ]:
for i in rg:
    im = oneband(i)
    im.save(i)


In [ ]:
# Open NumPy array with all images (should be 3 bands; number of images, height, width) 

x = np.array([np.array(Image.open(fname)) for fname in rg])

In [ ]:
x.shape

In [ ]:
# Flatten the images so they can be processed using Scikit.

x = x.reshape(1949, 462675)

In [ ]:
x.shape

In [ ]:
# Fit Kmeans with x clusters

kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(x)
kmeans.cluster_centers_.shape

In [ ]:
# Plot the result in subplots

fig, ax = plt.subplots(2, 5, figsize=(160, 60))
centers = kmeans.cluster_centers_.reshape(10, 775, 597)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

In [ ]:
centers.shape

In [ ]:
# Save the predicted model

filename = 'finalized_kmeans.sav'
joblib.dump(centers, filename)